In [2]:
from huggingface_hub import login
login()

Token has not been saved to git credential helper.


In [3]:
import pandas as pd
import numpy as np

In [4]:
df_train = pd.read_csv('/media/yanncauchepin/ExternalDisk/Datasets/ComputerVisionImages/digit_recognizer/train.csv')
df_test = pd.read_csv('/media/yanncauchepin/ExternalDisk/Datasets/ComputerVisionImages/digit_recognizer/test.csv')

In [5]:
distinct_labels = df_train['label'].value_counts()
print(pd.DataFrame(distinct_labels))
classes = len(df_train["label"].unique())

       count
label       
1       4684
7       4401
3       4351
9       4188
2       4177
6       4137
0       4132
4       4072
8       4063
5       3795


In [6]:
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

def evaluate_classifier(y_true, y_pred):
    f1 = f1_score(y_true, y_pred, average='weighted')
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    cm = confusion_matrix(y_true, y_pred)
    
    metrics_df = pd.DataFrame({
        'Metric': ['F1 Score', 'Precision', 'Recall'],
        'Value': [f1, precision, recall]
    })
    
    cm_df = pd.DataFrame(cm, columns=df_train["label"].unique(), index=df_train["label"].unique())
    
    return metrics_df, cm_df

In [7]:
y_train = np.array(df_train['label'])
X_train = np.array(df_train.drop('label', axis=1))
X_test = np.array(df_test)

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train, random_state=42)

In [9]:
X_train_norm = X_train / 255.0
X_valid_norm = X_valid / 255.0
X_test_norm = X_test / 255.0

In [36]:
from sklearn.neighbors import KNeighborsClassifier

knn_classifier = KNeighborsClassifier(n_neighbors=classes)
knn_classifier.fit(X_train_norm, y_train)

y_pred = knn_classifier.predict(X_valid_norm)

knn_classifier_assessement = evaluate_classifier(y_valid, y_pred)
print(knn_classifier_assessement)

(      Metric     Value
0   F1 Score  0.962301
1  Precision  0.963080
2     Recall  0.962381,      1    0    4    7    3    5    8    9    2    6
1  817    0    1    0    0    4    4    0    0    1
0    0  934    1    0    0    0    0    2    0    0
4    9   17  789    3    1    2    0   13    0    1
7    2    6    5  831    0   14    0    5    3    4
3    0   10    0    0  783    0    1    1    0   19
5    4    5    0   10    0  720    7    0    3   10
8    5    2    0    0    1    4  815    0    0    0
9    0   10    0    0    1    0    0  862    0    7
2    9   15    2   15    3   18    3    3  732   13
6    4    1    1    3   10    3    0   15    0  801)


In [38]:
from xgboost import XGBClassifier

xgb_classifier = XGBClassifier(n_estimators=1000, max_depth=4, learning_rate=0.1,  eval_metric='logloss')
xgb_classifier.fit(X_train_norm, y_train, verbose=1)

y_pred = xgb_classifier.predict(X_valid_norm)

xgb_classifier_assessement = evaluate_classifier(y_valid, y_pred)
print(xgb_classifier_assessement)

(      Metric     Value
0   F1 Score  0.975350
1  Precision  0.975385
2     Recall  0.975357,      1    0    4    7    3    5    8    9    2    6
1  812    0    1    0    2    1    6    0    2    3
0    0  930    2    3    0    0    0    0    2    0
4    0    0  815    4    5    0    0    8    2    1
7    3    2    7  832    3   10    0    3    3    7
3    1    4    0    0  794    0    2    1    1   11
5    4    1    1    7    1  732    6    0    4    3
8    0    1    1    0    0    5  817    0    3    0
9    0    3    4    2    2    1    0  861    1    6
2    3    1    0    2    0    6    3    1  788    9
6    3    2    0    3    4    2    0    9    3  812)


In [39]:
from catboost import CatBoostClassifier

catboost_classifier = CatBoostClassifier(iterations=1000, depth=4, learning_rate=0.1, verbose=2)
catboost_classifier.fit(X_train_norm, y_train)

y_pred = catboost_classifier.predict(X_valid_norm)

catboost_classifier_assessement = evaluate_classifier(y_valid, y_pred)
print(catboost_classifier_assessement)

0:	learn: 2.0962791	total: 397ms	remaining: 6m 36s
2:	learn: 1.8310850	total: 907ms	remaining: 5m 1s
4:	learn: 1.6302713	total: 1.45s	remaining: 4m 48s
6:	learn: 1.4751555	total: 1.96s	remaining: 4m 37s
8:	learn: 1.3460221	total: 2.44s	remaining: 4m 28s
10:	learn: 1.2434894	total: 2.94s	remaining: 4m 24s
12:	learn: 1.1528075	total: 3.46s	remaining: 4m 22s
14:	learn: 1.0777297	total: 3.91s	remaining: 4m 17s
16:	learn: 1.0118651	total: 4.4s	remaining: 4m 14s
18:	learn: 0.9546359	total: 4.84s	remaining: 4m 9s
20:	learn: 0.8999357	total: 5.3s	remaining: 4m 7s
22:	learn: 0.8518069	total: 5.8s	remaining: 4m 6s
24:	learn: 0.8097365	total: 6.33s	remaining: 4m 6s
26:	learn: 0.7709663	total: 6.89s	remaining: 4m 8s
28:	learn: 0.7421569	total: 7.36s	remaining: 4m 6s
30:	learn: 0.7083525	total: 7.89s	remaining: 4m 6s
32:	learn: 0.6796631	total: 8.36s	remaining: 4m 5s
34:	learn: 0.6590664	total: 8.73s	remaining: 4m
36:	learn: 0.6324410	total: 9.23s	remaining: 4m
38:	learn: 0.6080263	total: 9.72s	rem

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import f1_score
from tensorflow.keras.callbacks import Callback

class F1ScoreCallback(Callback):
    def __init__(self, validation_data):
        super().__init__()
        self.validation_data = validation_data

    def on_epoch_end(self, epoch, logs=None):
        val_data, val_labels = self.validation_data
        y_pred_proba = self.model.predict(val_data)
        y_pred = np.argmax(y_pred_proba, axis=1)
        f1 = f1_score(val_labels, y_pred, average='weighted')
        print(f"\nEpoch {epoch+1} - F1 Score: {f1:.4f}")
        logs['f1_score'] = f1  # To store in logs if needed

# Define the model
mlp_classifier = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_norm.shape[1],)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(10, activation='softmax')
])

mlp_classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Initialize callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
f1_callback = F1ScoreCallback(validation_data=(X_valid_norm, y_valid))

# Train the model with the F1 callback
mlp_classifier.fit(
    X_train_norm, 
    y_train, 
    epochs=20, 
    batch_size=64,
    validation_data=(X_valid_norm, y_valid),
    callbacks=[early_stopping, f1_callback]
)

# Evaluate on validation set
y_pred_proba = mlp_classifier.predict(X_valid_norm)
y_pred = np.argmax(y_pred_proba, axis=1)

# Compute final F1 score
from sklearn.metrics import classification_report
print(classification_report(y_valid, y_pred))

mlp_classifier_assessment = evaluate_classifier(y_valid, np.round(y_pred))
print(mlp_classifier_assessment)

/home/yanncauchepin/Git/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20


2024-11-01 20:21:25.092408: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 105369600 exceeds 10% of free system memory.


525/525 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7472 - loss: 0.8038 - val_accuracy: 0.9400 - val_loss: 0.2079
Epoch 2/20
525/525 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9292 - loss: 0.2314 - val_accuracy: 0.9555 - val_loss: 0.1463
Epoch 3/20
525/525 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9518 - loss: 0.1625 - val_accuracy: 0.9620 - val_loss: 0.1266
Epoch 4/20
525/525 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9590 - loss: 0.1356 - val_accuracy: 0.9646 - val_loss: 0.1170
Epoch 5/20
525/525 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9650 - loss: 0.1113 - val_accuracy: 0.9689 - val_loss: 0.1046
Epoch 6/20
525/525 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9714 - loss: 0.0942 - val_accuracy: 0.9692 - val_loss: 0.1033
Epoch 7/20
525/525 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9741 - loss: 0.0840 - val_accuracy: 0.9706 - val_loss: 0.1025
Epoch 8/20
525/525 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9759 - loss: 0.0752 - val_accuracy: 0.9714 - val_

In [ ]:
import torch
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, classification_report, f1_score
from datasets import Dataset

# Define the CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.model = nn.Sequential(
        
            nn.Conv2d(1, 16, kernel_size=5, padding=2, stride=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d((2, 2)), 
            
            nn.Conv2d(16, 32, kernel_size=3, padding=1, stride=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            
            nn.Conv2d(32, 64, kernel_size=3, padding=1, stride=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d((2, 2)),  
            
            nn.Conv2d(64, 128, kernel_size=3, padding=1, stride=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d((2, 2)),  
            
            nn.Flatten(), 
            
            nn.Linear(128 * 3 * 3, 100), 
            nn.BatchNorm1d(100),
            nn.ReLU(),
            nn.Dropout(0.25),
            
            nn.Linear(100, 64),
            nn.ReLU(),
            nn.Dropout(0.25),
            
            nn.Linear(64, 10)  
        )
    
    def forward(self, X):
        return self.model(X)

# Define transformation, dataset, and dataloaders
transform = transforms.Compose([
    transforms.Lambda(lambda x: x.reshape(28, 28).astype(np.float32)),
    transforms.ToTensor(),
])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Assuming X_train and X_valid are numpy arrays containing your data
X_train_transformed = torch.stack([transform(x) for x in X_train]).to(device)
X_valid_transformed = torch.stack([transform(x) for x in X_valid]).to(device)
y_train_tensor = torch.tensor(y_train).to(device)
y_valid_tensor = torch.tensor(y_valid).to(device)

train_dataset = TensorDataset(X_train_transformed, y_train_tensor)
valid_dataset = TensorDataset(X_valid_transformed, y_valid_tensor)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=8)

# Set up device, model, loss, and optimizer
cnn_classifier = CNN().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(cnn_classifier.parameters(), lr=0.001)

# Training function with F1 score tracking
def train_one_epoch(dataloader, model, loss_fn, optimizer, device):
    model.train()
    num_correct = 0
    track_loss = 0
    all_preds = []
    all_labels = []

    for i, (imgs, labels) in enumerate(dataloader):
        imgs, labels = imgs.to(device), labels.to(device)
        
        # Forward pass
        pred = model(imgs)
        loss = loss_fn(pred, labels)
        track_loss += loss.item()
        
        # Accumulate predictions and labels
        all_preds.extend(torch.argmax(pred, dim=1).cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        
        # Backward pass
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
    # Calculate metrics
    epoch_loss = track_loss / len(dataloader)
    epoch_acc = accuracy_score(all_labels, all_preds)
    epoch_f1 = f1_score(all_labels, all_preds, average="weighted")
    
    return epoch_loss, epoch_acc, epoch_f1

# Early stopping parameters
patience = 3
best_val_loss = float('inf')
early_stopping_counter = 0

# Main training loop with early stopping
num_epochs = 100
for epoch in range(num_epochs):
    print(f"\nEpoch {epoch + 1}/{num_epochs}")
    train_loss, train_acc, train_f1 = train_one_epoch(train_loader, cnn_classifier, criterion, optimizer, device)
    
    # Validation phase
    cnn_classifier.eval()
    val_loss = 0.0
    all_val_preds, all_val_labels = [], []
    with torch.no_grad():
        for inputs, labels in valid_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = cnn_classifier(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            
            all_val_preds.extend(torch.argmax(outputs, axis=1).cpu().numpy())
            all_val_labels.extend(labels.cpu().numpy())
    
    val_loss /= len(valid_loader)
    val_accuracy = accuracy_score(all_val_labels, all_val_preds)
    val_f1 = f1_score(all_val_labels, all_val_preds, average="weighted")
    
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}, Train F1 Score: {train_f1:.4f}")
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}, Validation F1 Score: {val_f1:.4f}")
    
    # Check for early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        early_stopping_counter = 0  # Reset counter if validation loss improves
    else:
        early_stopping_counter += 1
        print(f"Early stopping counter: {early_stopping_counter}/{patience}")
    
    if early_stopping_counter >= patience:
        print("Early stopping triggered.")
        break

# Final Evaluation on validation set
cnn_classifier.eval()
all_cnn_classifier_preds, all_cnn_classifier_labels = [], []

with torch.no_grad():
    for inputs, labels in valid_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = cnn_classifier(inputs)
        preds = torch.argmax(outputs, axis=1)
        
        all_cnn_classifier_preds.extend(preds.cpu().numpy())
        all_cnn_classifier_labels.extend(labels.cpu().numpy())

# Calculate final accuracy and classification report
accuracy = accuracy_score(all_cnn_classifier_labels, all_cnn_classifier_preds)
print(f"Final Validation Accuracy: {accuracy:.4f}")
print("Classification Report:\n", classification_report(all_cnn_classifier_labels, all_cnn_classifier_preds))

# Generate submission for test set
X_test_transformed = torch.stack([transform(x) for x in X_test]).to(device)
test_dataset = TensorDataset(X_test_transformed)
test_loader = DataLoader(test_dataset, batch_size=8)

cnn_classifier.eval()
test_preds = []

with torch.no_grad():
    for inputs in test_loader:
        inputs = inputs[0].to(device)
        outputs = cnn_classifier(inputs)
        preds = torch.argmax(outputs, axis=1)
        test_preds.extend(preds.cpu().numpy())

cnn_classifier_submission_df = pd.DataFrame({
    "ImageId": range(1, len(test_preds) + 1),
    "Label": test_preds
})

Dataset.from_pandas(cnn_classifier_submission_df).push_to_hub("yanncauchepin/kaggle_digitrecognizer_cnn_submission_df_2")


Epoch 1/100
Train Loss: 0.2530, Train Accuracy: 0.9273, Train F1 Score: 0.9273
Validation Loss: 0.0587, Validation Accuracy: 0.9808, Validation F1 Score: 0.9809

Epoch 2/100
Train Loss: 0.1173, Train Accuracy: 0.9659, Train F1 Score: 0.9659
Validation Loss: 0.0492, Validation Accuracy: 0.9849, Validation F1 Score: 0.9849

Epoch 3/100
Train Loss: 0.1021, Train Accuracy: 0.9711, Train F1 Score: 0.9711
Validation Loss: 0.0317, Validation Accuracy: 0.9902, Validation F1 Score: 0.9902

Epoch 4/100
Train Loss: 0.0806, Train Accuracy: 0.9770, Train F1 Score: 0.9770
Validation Loss: 0.0326, Validation Accuracy: 0.9918, Validation F1 Score: 0.9918
Early stopping counter: 1/3

Epoch 5/100
Train Loss: 0.0728, Train Accuracy: 0.9787, Train F1 Score: 0.9787
Validation Loss: 0.0301, Validation Accuracy: 0.9913, Validation F1 Score: 0.9913

Epoch 6/100
Train Loss: 0.0664, Train Accuracy: 0.9805, Train F1 Score: 0.9805
Validation Loss: 0.0514, Validation Accuracy: 0.9873, Validation F1 Score: 0.9873


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/28 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/307 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/yanncauchepin/kaggle_digitrecognizer_cnn_submission_df_2/commit/1b5be79d02dda63186a524ede508071ce1f3dcb0', commit_message='Upload dataset', commit_description='', oid='1b5be79d02dda63186a524ede508071ce1f3dcb0', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/yanncauchepin/kaggle_digitrecognizer_cnn_submission_df_2', endpoint='https://huggingface.co', repo_type='dataset', repo_id='yanncauchepin/kaggle_digitrecognizer_cnn_submission_df_2'), pr_revision=None, pr_num=None)

In [23]:
import os
from huggingface_hub import HfApi
import json

history_cnn = "custom_pytorch_cnn"
os.makedirs(history_cnn, exist_ok=True)

torch.save(cnn_classifier.state_dict(), os.path.join(history_cnn, "custom_pytorch_cnn.pth"))

config = {
    "model_type": "custom_pytorch_cnn",
    "num_labels": 10,
    "input_size": [1, 28, 28],
    "output_size": 10
}
with open(f"{history_cnn}/config.json", "w") as f:
    json.dump(config, f)
    
model_code = '''import torch
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.model = nn.Sequential(
        
            nn.Conv2d(1, 16, kernel_size=5, padding=2, stride=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d((2, 2)), 
            
            nn.Conv2d(16, 32, kernel_size=3, padding=1, stride=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            
            nn.Conv2d(32, 64, kernel_size=3, padding=1, stride=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d((2, 2)),  
            
            nn.Conv2d(64, 128, kernel_size=3, padding=1, stride=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d((2, 2)),  
            
            nn.Flatten(), 
            
            nn.Linear(128 * 3 * 3, 100), 
            nn.BatchNorm1d(100),
            nn.ReLU(),
            nn.Dropout(0.25),
            
            nn.Linear(100, 64),
            nn.ReLU(),
            nn.Dropout(0.25),
            
            nn.Linear(64, 10)  
        )
    
    def forward(self, X):
        return self.model(X)
'''
with open(f"{history_cnn}/custom_pytorch_cnn.py", "w") as f:
    f.write(model_code)
    
api = HfApi()
repo_id = "yanncauchepin/kaggle_digitrecognizer_cnn_model"
repo_url = api.create_repo(repo_id=repo_id, exist_ok=True)
api.upload_file(
    path_or_fileobj="/home/yanncauchepin/Git/Kaggle/DigitRecognizer/notebook/custom_pytorch_cnn/custom_pytorch_cnn.pth",
    path_in_repo="custom_pytorch_cnn.pth",
    repo_id=repo_id
)
api.upload_file(
    path_or_fileobj="/home/yanncauchepin/Git/Kaggle/DigitRecognizer/notebook/custom_pytorch_cnn/config.json",
    path_in_repo="config.json",
    repo_id=repo_id
)
api.upload_file(
    path_or_fileobj="/home/yanncauchepin/Git/Kaggle/DigitRecognizer/notebook/custom_pytorch_cnn/custom_pytorch_cnn.py",
    path_in_repo="custom_pytorch_cnn.py",
    repo_id=repo_id
)

custom_pytorch_cnn.pth:   0%|          | 0.00/898k [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/yanncauchepin/kaggle_digitrecognizer_cnn_model/commit/534026420a7ca029316f140f16fe2c97d7732f7e', commit_message='Upload custom_pytorch_cnn.py with huggingface_hub', commit_description='', oid='534026420a7ca029316f140f16fe2c97d7732f7e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/yanncauchepin/kaggle_digitrecognizer_cnn_model', endpoint='https://huggingface.co', repo_type='model', repo_id='yanncauchepin/kaggle_digitrecognizer_cnn_model'), pr_revision=None, pr_num=None)

In [55]:
# !pip install torchvision

  Using cached torch-2.5.1-cp312-cp312-manylinux1_x86_64.whl.metadata (28 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 1.1 MB/s eta 0:00:0000:0100:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 776.0 kB/s eta 0:00:000:0100:21m
  Attempting uninstall: torch
    Found existing installation: torch 2.5.0
    Uninstalling torch-2.5.0:
      Successfully uninstalled torch-2.5.0


In [10]:
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from pathlib import Path

# Specify the absolute path for saving tensors
external_path = Path("/media/yanncauchepin/ExternalDisk/RunningCode/digit_recognizer")
external_path.mkdir(parents=True, exist_ok=True)  # Create directory if it doesn't exist

# Define transformation pipeline
transform = transforms.Compose([
    transforms.Lambda(lambda x: torch.tensor(x.reshape(28, 28), dtype=torch.float32).unsqueeze(0)),
    transforms.Resize((224, 224)),
    transforms.Lambda(lambda x: x.expand(3, -1, -1)),
    transforms.Normalize((0.5,), (0.5,))
])

# Split and transform the dataset, saving each item to the external disk
def save_data_to_disk(data, labels, transform, path, prefix="train"):
    for i, (img, label) in enumerate(zip(data, labels)):
        img_tensor = transform(img)  # Apply transformations
        torch.save((img_tensor, torch.tensor(label)), path / f"{prefix}_{i}.pt")  # Save to disk

# Save the transformed training and validation data to external disk
save_data_to_disk(X_train, y_train, transform, external_path, prefix="train")
save_data_to_disk(X_valid, y_valid, transform, external_path, prefix="valid")

# Custom Dataset to load data from external disk on-the-fly
class DiskDataset(Dataset):
    def __init__(self, path, prefix="train"):
        self.path = path
        self.files = list(path.glob(f"{prefix}_*.pt"))  # Get all files with the prefix

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        data = torch.load(self.files[idx])  # Load tensor from disk
        return data

# Create DataLoaders for training and validation
train_dataset = DiskDataset(external_path, prefix="train")
valid_dataset = DiskDataset(external_path, prefix="valid")

train_loader = DataLoader(train_dataset, batch_size=3, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=3)

In [11]:
from transformers import ViTForImageClassification, AdamW
from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report

# Initialize ViT model
vit_classifier = ViTForImageClassification.from_pretrained(
    'google/vit-base-patch16-224-in21k', 
    num_labels=10
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vit_classifier.to(device)

# Training setup
optimizer = AdamW(vit_classifier.parameters(), lr=1e-5)
epochs = 3

# Training loop
for epoch in range(epochs):
    vit_classifier.train()
    train_loss = 0
    for batch in tqdm(train_loader):
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Forward pass
        outputs = vit_classifier(inputs, labels=labels)
        loss = outputs.loss
        train_loss += loss.item()
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    avg_train_loss = train_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{epochs}, Training Loss: {avg_train_loss:.4f}")

# Evaluation on validation set
vit_classifier.eval()
all_vit_classifier_preds, all_vit_classifier_labels = [], []

with torch.no_grad():
    for batch in valid_loader:
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)
        
        outputs = vit_classifier(inputs)
        preds = torch.argmax(outputs.logits, axis=1)
        
        all_vit_classifier_preds.extend(preds.cpu().numpy())
        all_vit_classifier_labels.extend(labels.cpu().numpy())

# Calculate accuracy and classification report
accuracy = accuracy_score(all_vit_classifier_labels, all_vit_classifier_preds)
print(f"Validation Accuracy: {accuracy:.4f}")
print("Classification Report:\n", classification_report(all_vit_classifier_labels, all_vit_classifier_preds))

# Additional assessment (if function `evaluate_classifier` is defined)
vit_classifier_assessment = evaluate_classifier(all_vit_classifier_labels, all_vit_classifier_preds)
print(vit_classifier_assessment)

vit_classifier.push_to_hub("yanncauchepin/kaggle_digitrecognizer_vit_model")

2024-11-02 23:02:56.570102: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-02 23:02:56.739895: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-02 23:02:56.783626: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-02 23:02:57.069157: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-02 23:02:58.956709: W tensorflow/compiler/tf2

Epoch 1/3, Training Loss: 0.6977


100%|██████████| 11200/11200 [1:34:05<00:00,  1.98it/s]


Epoch 2/3, Training Loss: 0.2149


100%|██████████| 11200/11200 [1:33:56<00:00,  1.99it/s]


Epoch 3/3, Training Loss: 0.1533
Validation Accuracy: 0.9501
Classification Report:
               precision    recall  f1-score   support

           0       0.99      0.97      0.98       827
           1       0.98      0.99      0.99       937
           2       0.98      0.91      0.94       835
           3       0.89      0.92      0.90       870
           4       0.99      0.94      0.97       814
           5       0.84      0.98      0.91       759
           6       0.99      0.97      0.98       827
           7       0.97      0.96      0.97       880
           8       0.95      0.91      0.93       813
           9       0.94      0.94      0.94       838

    accuracy                           0.95      8400
   macro avg       0.95      0.95      0.95      8400
weighted avg       0.95      0.95      0.95      8400

(      Metric     Value
0   F1 Score  0.950554
1  Precision  0.952667
2     Recall  0.950119,      1    0    4    7    3    5    8    9    2    6
1  799    

model.safetensors:   0%|          | 0.00/343M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/yanncauchepin/kaggle_digitrecognizer_vit_model/commit/00301fd101de7d31c7859b0db55ee00300b73253', commit_message='Upload ViTForImageClassification', commit_description='', oid='00301fd101de7d31c7859b0db55ee00300b73253', pr_url=None, repo_url=RepoUrl('https://huggingface.co/yanncauchepin/kaggle_digitrecognizer_vit_model', endpoint='https://huggingface.co', repo_type='model', repo_id='yanncauchepin/kaggle_digitrecognizer_vit_model'), pr_revision=None, pr_num=None)

In [ ]:
def save_data_to_disk(data, labels, transform, path, prefix="train"):
    for i, (img, label) in enumerate(zip(data, labels)):
        img_tensor = transform(img)  # Apply transformations
        
        # Save only the image tensor if the label is None
        if label is None:
            torch.save(img_tensor, path / f"{prefix}_{i}.pt")
        else:
            torch.save((img_tensor, torch.tensor(label)), path / f"{prefix}_{i}.pt")

save_data_to_disk(X_test, [None] * len(X_test), transform, external_path, prefix="test")  # Label is None for test

class DiskDataset(Dataset):
    def __init__(self, path, prefix="train", include_labels=True):
        self.path = path
        self.files = list(path.glob(f"{prefix}_*.pt"))
        self.include_labels = include_labels

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        data = torch.load(self.files[idx])
        
        # Return data accordingly (without labels for test data)
        if self.include_labels:
            img_tensor, label = data
            return img_tensor, label
        else:
            return data 

# Create a custom Dataset and DataLoader for the test set
test_dataset = DiskDataset(external_path, prefix="test", include_labels=False)
test_loader = DataLoader(test_dataset, batch_size=3)

vit_classifier.eval()
test_preds = []

with torch.no_grad():
    for batch in tqdm(test_loader):
        inputs = batch.to(device)  
        outputs = vit_classifier(inputs)
        preds = torch.argmax(outputs.logits, axis=1)
        test_preds.extend(preds.cpu().numpy())

vit_classifier_submission_df = pd.DataFrame({
    "ImageId": range(1, len(test_preds) + 1),
    "Label": test_preds
})
from datasets import Dataset
Dataset.from_pandas(vit_classifier_submission_df).push_to_hub("yanncauchepin/kaggle_digitrecognizer_vit_submission_df")


  0%|          | 0/9334 [00:00<?, ?it/s]/tmp/ipykernel_25585/3633612418.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(self.files[idx])
100%|████████

AttributeError: type object 'Dataset' has no attribute 'from_pandas'